In [1]:
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from flair.data import Sentence
from flair.models import TextClassifier

### Data Labelling

In [5]:
# Textblob
textblob_label = pd.read_csv('../1_Preprocessed_Data/abc_preprocessed.csv')

for index, row in textblob_label.iterrows():
    text = row['headline_text']
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity

    if sentiment > 0:
        label = 'positive'
    elif sentiment < 0:
        label = 'negative'
    else:
        label = 'neutral'

    textblob_label.at[index, 'textblob_label'] = label

textblob_label.to_csv('../2_Labelled_Data/textblob_label.csv', index=False) 
textblob_label.head()


publish_date                                      headline_text  \
0   2019-01-24  [('malaysia', 'appoint', 'new'), ('appoint', '...   
1   2019-01-21  [('kurti', 'patterson', 'rush'), ('patterson',...   
2   2019-01-31  [('katherin', 'darwin', 'lash'), ('darwin', 'l...   
3   2019-01-26  [('spirit', 'matthew', 'flinder'), ('matthew',...   
4   2019-01-31  [('nsw', 'govern', 'negoti'), ('govern', 'nego...   

  textblob_label  
0       positive  
1        neutral  
2        neutral  
3        neutral  
4       positive

In [6]:
#Vader
vader_label = pd.read_csv('../1_Preprocessed_Data/abc_preprocessed.csv')

analyzer = SentimentIntensityAnalyzer()

for index, row in vader_label.iterrows():
    text = row['headline_text'] 

    sentiment_scores = analyzer.polarity_scores(text)

    if sentiment_scores['compound'] > 0:
        label = 'positive'
    elif sentiment_scores['compound'] < 0:
        label = 'negative'
    else:
        label = 'neutral'

    vader_label.at[index, 'vader_label'] = label  

vader_label.to_csv('../2_Labelled_Data/vader_label.csv', index=False) 
vader_label.head()

publish_date                                      headline_text vader_label
0   2019-01-24  [('malaysia', 'appoint', 'new'), ('appoint', '...     neutral
1   2019-01-21  [('kurti', 'patterson', 'rush'), ('patterson',...     neutral
2   2019-01-31  [('katherin', 'darwin', 'lash'), ('darwin', 'l...     neutral
3   2019-01-26  [('spirit', 'matthew', 'flinder'), ('matthew',...    positive
4   2019-01-31  [('nsw', 'govern', 'negoti'), ('govern', 'nego...     neutral

In [3]:
# BERT
bert_label = pd.read_csv('../1_Preprocessed_Data/abc_preprocessed.csv')

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = 3
learning_rate = 2e-5
num_epochs = 5
batch_size = 32

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

predictions = []

for index, row in bert_label.iterrows():
    text = row['headline_text']

    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    encoded_input = {key: tensor.to(device) for key, tensor in encoded_input.items()}

    with torch.no_grad():
        output = model(**encoded_input)

    predicted_label = torch.argmax(output.logits).item()
    predictions.append(predicted_label)

labels = ['negative', 'neutral', 'positive']
predicted_sentiments = [labels[prediction] for prediction in predictions]

bert_label['bert_label'] = predicted_sentiments

bert_label.to_csv('../2_Labelled_Data/bert_label.csv', index=False)
bert_label.head()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

publish_date                                      headline_text bert_label
0   2019-01-20  [('thousand', 'attend', 'woman'), ('attend', '...   positive
1   2019-01-22  [('former', 'socceroo', 'craig'), ('socceroo',...   positive
2   2019-01-14  [('medium', 'attack', 'outsid'), ('attack', 'o...   positive
3   2019-01-28  [('ambul', 'offic', 'dy'), ('offic', 'dy', 'at...   positive
4   2019-01-25  [('florida', 'offici', 'resign'), ('offici', '...   positive

In [8]:
#Flair
flair_label = pd.read_csv('../1_Preprocessed_Data/abc_preprocessed.csv')

classifier = TextClassifier.load('en-sentiment')

flair_label['flair_label'] = ""

for i, headline in enumerate(flair_label['headline_text']):
    sentence = Sentence(headline)
    classifier.predict(sentence)
    label = sentence.labels[0].value.lower()
    flair_label.at[i, 'flair_label'] = label

flair_label.to_csv('../2_Labelled_Data/flair_label.csv', index=False) 
flair_label.head()

publish_date                                      headline_text flair_label
0   2019-01-24  [('malaysia', 'appoint', 'new'), ('appoint', '...    negative
1   2019-01-21  [('kurti', 'patterson', 'rush'), ('patterson',...    negative
2   2019-01-31  [('katherin', 'darwin', 'lash'), ('darwin', 'l...    negative
3   2019-01-26  [('spirit', 'matthew', 'flinder'), ('matthew',...    negative
4   2019-01-31  [('nsw', 'govern', 'negoti'), ('govern', 'nego...    negative